In [ ]:
from torch.optim.optimizer import Optimizer
import torch
import torchvision 
from torchvision import transforms
import torch.nn.functional as F

torch.cuda.is_available()


#define the AlexNet(a common CNN)
class AlexNet(torch.nn.Module):
  def __init__(self,num_classes):
    super(AlexNet,self).__init__()
    self.num_classes=num_classes

    self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            torch.nn.BatchNorm2d(96),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 3, stride = 2))
    self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 3, stride = 2))
    self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(384),
            torch.nn.ReLU())
    self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(384),
            torch.nn.ReLU())
    self.layer5 = torch.nn.Sequential(
            torch.nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 3, stride = 2))
    self.fc = torch.nn.Sequential(
          torch.nn.Dropout(0.5),
          torch.nn.Linear(9216, 4096),
          torch.nn.ReLU())
    self.fc1 = torch.nn.Sequential(
          torch.nn.Dropout(0.5),
          torch.nn.Linear(4096, 4096),
          torch.nn.ReLU())
    self.fc2= torch.nn.Sequential(
         torch.nn.Linear(4096, num_classes))
        
  def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        
        out = out.reshape(out.size(0), -1)
        
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out      
  #define the loss function



In [ ]:
def get_loss_func():
   loss_func=torch.nn.CrossEntropyLoss()
   return loss_func
 #define the optimizer #ways to get min loss?
def get_opt(net,lr,wd,momentum):
   optimizer=torch.optim.SGD(net.parameters(),lr=lr,weight_decay=wd,momentum=momentum)
   return optimizer

In [ ]:
import torch
print(torch.cuda.device_count())
torch.cuda.is_available()
#make sure there is gpu available

1


True

In [ ]:
def train(net,data_loader,optimizer,cost_function,device='cuda:0'):
    samples=0.
    cumulative_loss=0.
    cumulative_accuracy=0.
    net.train()       ###train指令
    for batch_idx, (inputs, targets) in enumerate(data_loader): # Load data into GPU
     torch.cuda.is_available()
    
     net=net.to(device)
     inputs = inputs.to(device) 
     targets = targets.to(device) 
     
     outputs = net(inputs)
     loss=loss_func(outputs,targets)  #forward pass
     
     loss.backward()   
     optimizer.step()   #backward and optimize
     optimizer.zero_grad()
     samples+=inputs.shape[0]
     cumulative_loss+=loss.item()
     _,predicted=outputs.max(1)
     cumulative_accuracy+=predicted.eq(targets).sum().item()
    return cumulative_loss/samples,cumulative_accuracy/samples*100

In [ ]:
def test(net,data_loader,cost_function,device='cuda:0'):
    sample=0.
    cumulative_loss=0.
    cumulative_accuracy=0.
    device=torch.device('cuda'if torch.cuda.is_available()else'cpu')
    torch.device('cuda:0')
    net.eval()#test and train:diff layers
    with torch.no_grad():
     for batch_idx,(inputs,targets) in enuerate(data_loader):#load data into gpu
       torch.cuda.is_available()
       net=net.to(device)
       inputs=inputs.to(device)
       targets=targets.to(device) #forward pass?
       outputs=net(inputs)#apply the loss
      
       _,predicted=outputs.max(1)
       cumulative_accuracy+=predicted.eq(targets).sum().item()
    return cumulative_loss/samples,cumulative_accuracy/samples*100

In [ ]:
def get_data(batch_size,test_batch_size=256):
    transform= list()#converts numpy to pytorch tensor
    transform.append(transforms.Resize((227,227)))
    transform.append(transforms.ToTensor())
    transform.append(transforms.Normalize(mean=[0.5],std=[0.5]))
    transform=transforms.Compose(transform)#把transform结合到一个
    #load data
    full_training_data=torchvision.datasets.CIFAR10('./data', train=True, transform=transform, download=True) 
    test_data=torchvision.datasets.CIFAR10('./data', train=False, transform=transform, download=True) 
    #create train and validation splits#sample
    num_samples=len(full_training_data)
    training_samples = int(num_samples*0.8+1) 
    validation_samples = num_samples - training_samples
    training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples,validation_samples])
    #initialize dataloaders
    train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(validation_data, test_batch_size, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

In [ ]:
    batch_size=128
    device='cuda:0'
    learning_rate=0.1
    weight_decay=0.000001
    momentum=0.9
    num_epochs=50

In [ ]:

    train_loader,val_loader,test_loader=get_data(batch_size)
    net=AlexNet(num_classes=10)
    optimizer = get_opt(net, learning_rate, weight_decay, momentum)
    loss_func=get_loss_func() 
    torch.cuda.is_available()
    for epochs in range(num_epochs):
      train_loss, train_accuracy = train(net, train_loader, optimizer, loss_func)
      val_loss, val_accuracy = test(net, val_loader, loss_func) 
      print('Epoch: {:d}'.format(e+1))
      print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss,train_accuracy))
      print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss,val_accuracy))
      print('________________')


100%|██████████| 170498071/170498071 [00:13<00:00, 12620130.04it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


NameError: ignored